In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexSets

In [ ]:
gcs = GraphOfConvexSets()

f1 = gcs.add_vertex("f1")
xf1 = f1.add_variable(2)
cf1 = np.array([3, 2.5])
f1.add_constraint(cp.norm(xf1 - cf1, np.inf) <= .5)

f2 = gcs.add_vertex("f2")
xf2 = f2.add_variable(2)
cf2 = np.array([3, -1])
Df2 = np.diag([2, .5])
f2.add_constraint(cp.norm(Df2 @ (xf2 - cf2), np.inf) <= 1)

u1 = gcs.add_vertex("u1")
xu1 = u1.add_variable(2)
cu1 = np.array([0, .5])
u1.add_constraint(cp.norm(xu1 - cu1, 2) <= .5)

u2 = gcs.add_vertex("u2")
xu2 = u2.add_variable(2)
cu2 = np.array([0, -1])
u2.add_constraint(cp.norm(xu2 - cu2, 2) <= .5)

u3 = gcs.add_vertex("u3")
xu3 = u3.add_variable(2)
cu3 = np.array([0, -2.5])
u3.add_constraint(cp.norm(xu3 - cu3, 2) <= .5)

In [ ]:
facilities = [f1, f2]
users = [u1, u2, u3]
for facility in facilities:
    for user in users:
        edge = gcs.add_edge(facility, user)
        edge.add_cost(cp.norm(facility.variables[0] - user.variables[0], 2))

In [ ]:
gcs.graphviz()

In [ ]:
prob = gcs.solve_facility_location()
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.gca().set_aspect('equal')
plt.axis('off')
gcs.plot_2d()
gcs.plot_subgraph_2d()
# plt.savefig('facility_location.pdf')

# From ILP

In [ ]:
ilp_constraints = []
yv = gcs.vertex_binaries()
ye = gcs.edge_binaries()

for i, v in enumerate(gcs.vertices):
    inc_edges = gcs.incoming_indices(v)
    if len(inc_edges) == 0:
        ilp_constraints.append(yv[i] <= 1)
    else:
        ilp_constraints.append(yv[i] == 1)
        ilp_constraints.append(sum(ye[inc_edges]) == 1)
        
for k, edge in enumerate(gcs.edges):
    i = gcs.vertex_index(edge.tail)
    ilp_constraints.append(yv[i] >= ye[k])

In [ ]:
prob = gcs.solve_from_ilp(ilp_constraints)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)